In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
drive="/content/gdrive/My Drive/AMEX"

In [3]:
import pandas as pd
import numpy as np
import matplotlib
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.callbacks import TensorBoard
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

Using TensorFlow backend.


In [5]:
train = pd.read_csv(drive+"/train_ready_for_model.csv")
#train.drop(columns='DateTime',inplace=True)
train.head()

,product_A,product_B,product_C,product_D,product_E,product_F,product_G,product_H,product_I,product_J,...,Weekday_name_Sunday,Weekday_name_Thursday,Weekday_name_Tuesday,Weekday_name_Wednesday,session_id,user_id,is_click,pc_interest_count,day_view_count,day_interest_count
0,0,0,1,0,0,0,0,0,0,0,...,1,0,0,0,140690,858557,0.0,0.0,0.0,0.0
1,0,0,1,0,0,0,0,0,0,0,...,1,0,0,0,333291,243253,0.0,0.0,0.0,0.0
2,0,0,1,0,0,0,0,0,0,0,...,1,0,0,0,129781,243253,0.0,0.0,0.0,0.0
3,0,0,0,0,0,0,0,0,1,0,...,1,0,0,0,464848,1097446,0.0,2.0,1.0,0.0
4,0,0,1,0,0,0,0,0,0,0,...,1,0,0,0,90569,663656,0.0,3.0,0.0,0.0


In [6]:
test = pd.read_csv(drive+"/test_ready_for_model.csv")
#test.drop(columns='DateTime',inplace=True)
test.head()

,product_A,product_B,product_C,product_D,product_E,product_F,product_G,product_H,product_I,product_J,...,Weekday_name_Sunday,Weekday_name_Thursday,Weekday_name_Tuesday,Weekday_name_Wednesday,session_id,user_id,is_click,pc_interest_count,day_view_count,day_interest_count
0,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,411705,732573,NaN,1.0,0.0,0.0
1,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,208263,172910,NaN,3.0,0.0,0.0
2,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,239450,172910,NaN,3.0,0.0,0.0
3,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,547761,557318,NaN,0.0,0.0,0.0
4,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,574275,923896,NaN,0.0,0.0,0.0


In [7]:
result = test[["session_id"]]
result.head()

,session_id
0,411705
1,208263
2,239450
3,547761
4,574275


In [0]:
#Define target and ID columns:
target = 'is_click'
IDcol = ['session_id']
colorder=['session_id','is_click']
from sklearn import model_selection, metrics
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.metrics import accuracy_score
import time

def modelfit(alg, dtrain, dtest, predictors, target, IDcol, filename):
    #Fit the algorithm on the data
    t1=time.time()
    alg.fit(dtrain[predictors], dtrain[target])
        
    #Predict training set:
    dtrain_predictions = alg.predict(dtrain[predictors])
    kfold = model_selection.KFold(n_splits=10, random_state=7)

    #Perform cross-validation:
    cv_train_score = model_selection.cross_val_score(alg, dtrain[predictors], dtrain[target], cv=kfold, scoring='accuracy')
    #fpr, tpr, tresholds = roc_curve(dtrain[target], alg.predict(dtrain[predictors]))
    #cv_train_score = np.sqrt(np.abs(cv_train_score))
    
    #Print model report:
    print("\nModel Report")
    print(classification_report(dtrain[target].values, dtrain_predictions))
    accuracy=accuracy_score(dtrain[target].values, dtrain_predictions)
    print("Accuracy: %.2f%%" % (accuracy * 100.0))
    #print ('AUC:', auc(fpr, tpr))
    #print('ROC_AUC Score: ',roc_auc_score(dtrain[target], alg.predict(dtrain[predictors])))
    #print("Train CV score : ", cv_train_score)
    #print("Train RMSE : %.4g" % np.sqrt(metrics.mean_squared_error(dtrain[target].values, dtrain_predictions)))
    
    #Predict on testing data:
    dtest[target] = alg.predict(dtest[predictors])
    
    #Export submission file:
    IDcol.append(target)
    submission = pd.DataFrame({ x: dtest[x] for x in IDcol})
    submission=submission[colorder]
    submission.to_csv(filename, index=False)
    t2=time.time()
    t_linalg_lstsq = float(t2-t1)
    print("Time taken: {} seconds".format(t_linalg_lstsq))

In [24]:
from sklearn.linear_model import LogisticRegression
predictors = [x for x in train.columns if x not in [target]+IDcol]
alg1 = LogisticRegression()
modelfit(alg1, train, test, predictors, target, IDcol, drive+'/logreg.csv')


Model Report
             precision    recall  f1-score   support

        0.0       0.93      1.00      0.97    431960
        1.0       0.00      0.00      0.00     31331

avg / total       0.87      0.93      0.90    463291

ROC_AUC Score:  0.5
Time taken: 22.594197034835815 seconds


In [21]:
from sklearn.tree import DecisionTreeClassifier
alg1 = DecisionTreeClassifier(criterion='entropy',max_depth=30, min_samples_split=50,min_samples_leaf=25)
modelfit(alg1, train, test, predictors, target, IDcol, 'res7.csv')


Model Report
             precision    recall  f1-score   support

        0.0       0.93      1.00      0.97    431960
        1.0       0.00      0.00      0.00     31331

avg / total       0.87      0.93      0.90    463291

AUC: 0.5
ROC_AUC Score:  0.5
Time taken: 150.12157893180847 seconds


In [0]:
from sklearn.ensemble import RandomForestClassifier
alg1 = RandomForestClassifier(n_estimators=1000,max_depth=30,min_samples_split=2,min_samples_leaf=1,n_jobs=-1)
modelfit(alg1, train, test, predictors, target, IDcol, drive+'/Rande1000.csv')

In [0]:
from sklearn.ensemble import RandomForestClassifier
alg1 = alg1 = RandomForestClassifier(n_estimators=200,max_depth=30,min_samples_split=2,min_samples_leaf=1,n_jobs=-1)
modelfit(alg1, train, test, predictors, target, IDcol, drive+'/Rande500.csv')

In [30]:
from xgboost import XGBClassifier


alg1=XGBClassifier(max_depth=15,n_jobs=-1)
modelfit(alg1, train, test, predictors, target, IDcol, drive+'/XGB.csv')


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this wi


Model Report
             precision    recall  f1-score   support

        0.0       0.93      1.00      0.97    431960
        1.0       0.99      0.02      0.04     31331

avg / total       0.94      0.93      0.90    463291

Accuracy: 93.39%


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


Time taken: 6664.773607492447 seconds
